### 1. 데이터 준비

In [ ]:
import torch  ##PyTorch 라이브러리 임포트 ( 딥러닝 프레임워크 )
from torch.utils.data import Dataset, DataLoader  ## 데이터셋과 데이터로더를 위한 모듈 임포트
import pandas as pd  ## 데이터프레임 처리를 위한 pandas 라이브러리 임포트
import numpy as np  ## 배열 연산을 위한 numpy 라이브러리 임포트

## ASL 데이터셋 클래스 정의 (PyTorch의 Dataset 클래스를 상속)
class ASL_Dataset(Dataset):  
  def __init__(self, path):
    super(ASL_Dataset, self).__init__()  ## 부모 클래스 ( Dataset )의 초기화 메서드 호출

## CSV 파일을 읽어 데이터프레임으로 로드
    df = pd.read_csv(path)
    self.y = df['label']  ## 'label' 열을 추출 -> 타겟 변수 ( y )로 설정
    del df['label']  ## 데이터프레임에서 'label' 열 삭제 ( 입력 데이터만 남김 )
    self.x = df.values  ## 나머지 데이터를 numpy 배열로 변환하여 입력 변수 ( x )로 설정


## 데이터에서 특정 인덱스의 샘플을 반환하는 메서드
  def __getitem__(self, index):
    x_sample = np.uint8( self.x[index].reshape(28,28) )  ## 입력 데이터를 28X28 크기의 이미지로 재구성 ( uint8 타입으로 변경 )
    x_tensor = torch.tensor ( x_sample[np.newaxis,:,:] ).float()  ## 단일 채널 ( 1차원 )을 추가하여 텐서로 변환 ( 형식 : [1, 28, 28]), float 타입으로 변환
    y_tensor = torch.tensor ( self.y[index] )  ## 해당 인덱스의 레이블을 텐서로 변환
    return x_tensor, y_tensor  ## 입력 텐서와 레이블 텐서를 튜플로 반환


## 데이터셋의 전체 길이 ( 샘플 수 )를 반환하는 메서드
  def __len__(self):
    return self.x.shape[0]  ## 입력 데이터의 행 개수( 샘플 수 ) 반환환

In [ ]:
## 훈련 데이터셋 객체 생성 ( sign_mnist_train.csv 파일을 기반으로 ASL_Dataset 인스턴스 생성 )
train_data = ASL_Dataset("sign_mnist_train.csv")

## 검증 데이터셋 객제 생성 ( sign_mnist_valid.csv 파일을 기반으로 ASL_Dataset 인스턴스 생성 )
valid_data = ASL_Dataset("sign_mnist_valid.csv")

## 훈련 데이터로드 생성
## - dataset : 생성할 데이터셋 ( train_data )
## - batch_size :  한 번에 처리할 샘플 수 ( 1000개 )
## - shuffle : 데이터 무작위로 섞을지 여부 ( True로 설정하여 섞음 )
train_loader = DataLoader(dataset=train_data, batch_size=1000, shuffle=True)

## 검증 데이터로더 생성
## - datastet : 사용할 데이터셋 ( valid_data )
## - batch_size : 한 번에 처리할 샘플 수 ( 1000개 )
## - shuffle : 데이터 무작위로 섞을지 여부 ( False로 설정하여 섞지 않음 )
valid_loader = DataLoader(dataset=valid_data, batch_size=1000, shuffle=False)

### 2. 뉴럴네트워크 모델링

In [ ]:
import torch.nn as nn
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      ## 첫 번째 합성곱 층 : 입력 채널 1 , 출력체널 16 , 커널 크기 3x3 , 패딩 1
      self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
      ## 두 번째 합성곱 층 : 입력 채널 16 , 출력체널 32 , 커널 크기 3x3 , 패딩 1
      self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
      ## 세 번째 합성곱 층 : 입력 채널 32 , 출력체널 32 , 커널 크기 3x3 , 패딩 1
      self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
      ## 활성화 함수 : ReLU ( 음수를 0으로 바꿔줌 )
      self.relu = nn.ReLU()
      ## 최대 풀링 : 2X2 크기로 특징 맵 크기를 줄임
      self.pool = nn.MaxPool2d(2,2)
      ## 완전 연결 층 : 입력 크기 7*7&32 , 출력 크기 24
      self.fc = nn.Linear(7*7*32, 24)

  def forward(self, x):
    ## 첫 번째 합성곱 층을 통과하고 ReLU 적용
    x = self.relu( self.conv1(x) )
    ## 풀링을 통해 크기 감소 : 14X14X16
    x = self.pool(x)   # 14X14X16
    ## 두 번째 함성곱 층을 통과하고 ReLU 적용
    x = self.relu( self.conv2(x) )
    ## 풀링을 통해 크기 감소 7X7X32
    x = self.pool(x)   # 7X7X32
    ## 세번째 합성곱 층을 통과하고 ReLU 적용 : 7X7X32
    x = self.relu( self.conv3(x) )   # 7x7x32

    ## 텐서를 1차원으로 펼침 : 배치 크기 X ( 7*7*32 )
    x = x.view(-1, 7*7*32)
    ## 완전 연결 층을 통과하여 최종 출력 생성
    x = self.fc(x)
    return x

In [ ]:
## Net 클래스의 인스턴스를 생성하여 convnet 변수에 할당
convnet = Net()

### 3.손실함수와 최적화함수의 지정

In [ ]:
## 손실 함수 정의 : 크로스 엔트로피 손실 ( 분류 문제에 적합 )
loss_function = nn.CrossEntropyLoss()
## 최적화 도구 정의 : Adam 알고리즘 사용 ,학습률 0.001로 설정 , convnet의 파라미터를 최적화
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. 뉴럴네트워크 학습

In [ ]:
## 5번의 에포크 동안 학습 반복
for epoch in range(5):
    ## 손실 값을 저장할 변수 초기화
    loss_val = 0
    ## 데이터 로더에서 배치 단위로 데이터를 가져와 반복
    for itr, data in enumerate(train_loader):
        ## 옵티마이저의 기울기 초기화 ( 이전 기울기 제거 )
        optimizer.zero_grad()
        
        ##데이터에서 입력과 레이블 분리
        inputs, labels = data
        ##신경망을 통해 예측값 계싼
        pred = convnet(inputs)
        
        ## 예측값과 실제 레이블 간의 손실 계산
        loss = loss_function(pred, labels)

        ## 손실에 대한 기울기 계산 ( 역전파 )
        loss.backward()

        ## 기울기를 바탕으로 파라미터 업데이트
        optimizer.step()

        ## 이번 배치의 손실 값을 누적
        loss_val += loss.item()

    ## 에포크 종료 후 손실 값 출력    
    print("Epoch:", epoch+1, " , Loss:", loss_val)

Epoch: 1  , Loss: 139.78275990486145
Epoch: 2  , Loss: 41.181666135787964
Epoch: 3  , Loss: 12.290689006447792
Epoch: 4  , Loss: 4.458873055875301
Epoch: 5  , Loss: 1.911327749490738


### 5. 뉴럴네트워크 평가

In [ ]:
## 예측값과 실제 레이블을 저장할 리스트 초기화
pred_list = []
label_list = []
## 검증 데이터 로더에서 배치 단위로 데이터를 가져와 반복
for itr, data in enumerate(valid_loader):
  ## 데이터에서 입력과 레이블 분리
  inputs, labels = data
  ## 신경망을 통해 검증 데이터에 대한 예측값 계산
  pred_test = convnet(inputs)
  ## 각 예측값에서 가장 높은 확률을 가진 카테고리 선택
  pred_category = torch.argmax(pred_test, dim=1)

  ## 예측 카테고리와 레이블을 리스트에 추가
  pred_list = pred_list + list(pred_category)
  label_list = label_list + list(labels)

## 예측값과 실제 레이블 간 일치 여부를 계산하여 정확도 도출
accu = np.mean( np.array(pred_list) ==np.array(label_list) )
## 검증 데이터에 대한 정확도 출력
print("Test accuracy: ", accu)

Test accuracy:  0.8623814835471277
